In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
def read_file(path,extension,blank_firstrow=False):
    if extension == 'csv':
        if blank_firstrow:
            df = pd.read_csv(path, sep = ';', skiprows = 1, escapechar='\n')
        else:
            df = pd.read_csv(path, sep = ';', escapechar='\n')
    elif extension == 'txt':
        df = pd.read_csv(path, sep = '\t')
    elif extension == 'xls':
        df = pd.read_excel(path, skiprows = 1)
    return df

In [3]:
def remove_rank_cols(df):
    col_names = list(df)
    del_list = []
    for item in col_names:
        if 'rank' in item.lower():
            del_list.append(item)

    df.drop(del_list, axis=1, inplace=True)
    return df

In [4]:
def clean_states_text(states):    
    cleaned = []
    invalidChars = string.punctuation

    for word in states:
        word= word.strip()
        for char in word:
            if(char in invalidChars):
                word= word.replace(char, '')
                word= word.replace(' ', '_')
            else:
                word= word.replace(' ', '_')
        cleaned.append(word)

    return cleaned

In [5]:
def index_sort(df):
    df.set_index('State', inplace=True)
    df.sort_index(axis = 0, inplace= True)
    return df

In [6]:
def drop_duplicate_rows(df):
    if df.index.has_duplicates:
        df = df.drop_duplicates(subset = ["State"])
    return df

In [7]:
def remove_summary_row(df):
    if "United_States" in df.index:
        df.drop('United_States', inplace = True)
    return df

In [8]:
df_edu = read_file('../data/csv/education.csv','csv',True)
df_reg = read_file('../data/csv/region.txt','txt')
df_area = read_file('../data/csv/area.csv','csv')
df_exp = read_file('../data/csv/life_expectancy.csv','csv')
df_crime = read_file('../data/csv/crime.csv','csv')
df_income = read_file('../data/csv/income.xls','xls',True)

### Cleaning Education file

In [9]:
df_edu.head()

,"State,federal district,or territory",% High school graduateor higher,High School rank,% Bachelor's degreeor higher,Bachelor's rank,% Advanced degree,Advanced rank
0,Montana,93.0%,1.0,30.7%,21.0,10.1%,33.0
1,New Hampshire,92.8%,2.0,36.0%,9.0,13.8%,10.0
2,Minnesota,92.8%,3.0,34.8%,11.0,11.8%,18.0
3,Wyoming,92.8%,4.0,26.7%,41.0,9.3%,39.0
4,Alaska,92.4%,5.0,29.0%,28.0,10.4%,29.0


In [10]:
df_edu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   State,federal district,or territory  52 non-null     object 
 1   % High school graduateor higher      52 non-null     object 
 2   High School rank                     52 non-null     float64
 3   % Bachelor's degreeor higher         52 non-null     object 
 4   Bachelor's rank                      52 non-null     float64
 5   % Advanced degree                    52 non-null     object 
 6   Advanced rank                        52 non-null     float64
dtypes: float64(3), object(4)
memory usage: 3.0+ KB


In [11]:
# Remove rank columns
df_edu = remove_rank_cols(df_edu)

# Rename columns
df_edu.rename(columns = {'State,federal district,or territory':'State','% High school graduateor higher':'HS_Grad','% Bachelor\'s degreeor higher':'BS_Grade','% Advanced degree':'Adv_Degree' }, inplace = True)

# Convert object types to state
df_edu['HS_Grad'] = pd.to_numeric(df_edu['HS_Grad'].str.replace('%',''))  
df_edu['BS_Grade'] = pd.to_numeric(df_edu['BS_Grade'].str.replace('%',''))  
df_edu['Adv_Degree'] = pd.to_numeric(df_edu['Adv_Degree'].str.replace('%','')) 

# Remove invalid characters and replace whitespace in States with underscore
state_list = df_edu['State'].tolist()
df_edu['State'] = clean_states_text(state_list)

# Index and sort by State
df_edu = index_sort(df_edu)

# Drop duplicate index
df_edu = drop_duplicate_rows(df_edu)

# Remove summary row
df_edu = remove_summary_row(df_edu)

In [12]:
df_edu.head()

,HS_Grad,BS_Grade,Adv_Degree
State,,,
Alabama,85.3,24.5,9.1
Alaska,92.4,29.0,10.4
Arizona,82.1,28.4,10.7
Arkansas,85.6,22.0,7.9
California,82.5,32.6,12.2


In [13]:
df_edu.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Alabama to Wyoming
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HS_Grad     51 non-null     float64
 1   BS_Grade    51 non-null     float64
 2   Adv_Degree  51 non-null     float64
dtypes: float64(3)
memory usage: 1.6+ KB


### Cleaning Region file

In [14]:
df_reg.head()

,Name,Abb,Region,Division
1,Alabama,AL,South,East South Central
2,Alaska,AK,West,Pacific
3,Arizona,AZ,West,Mountain
4,Arkansas,AR,South,west south central
5,California,CA,West,Pacific


In [15]:
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      50 non-null     object
 1   Abb       50 non-null     object
 2   Region    50 non-null     object
 3   Division  50 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


In [16]:
df_reg.shape

(50, 4)

In [17]:
# Rename columns
df_reg.rename(columns = {'Name':'State' }, inplace = True)

# Remove invalid characters and replace whitespace in States with underscore
state_list = df_reg['State'].tolist()
df_reg['State'] = clean_states_text(state_list)

# Index and sort by State
df_reg = index_sort(df_reg)

In [18]:
df_reg.head()

,Abb,Region,Division
State,,,
Alabama,AL,South,East South Central
Alaska,AK,West,Pacific
Arizona,AZ,West,Mountain
Arkansas,AR,South,west south central
California,CA,West,Pacific


### Cleaning Area file

In [19]:
df_area.head()

,State,TotalRank,TotalSqMi,TotalKmQ,LandRank,LandSqMi,LandKmQ,LandPer,WaterRank,WaterSqMi,WaterKmQ,WaterPer
0,Alaska,1,665384.04,1723337,1,570640.95,1477953,85.76,1,94743.10,245384,14.24
1,Texas,2,268596.46,695662,2,261231.71,676587,97.26,8,7364.75,19075,2.74
2,California,3,163694.74,423967,3,155779.22,403466,95.16,6,7915.52,20501,4.84
3,Montana,4,147039.71,380831,4,145545.80,376962,98.98,26,1493.91,3869,1.02
4,New Mexico,5,121590.30,314917,5,121298.15,314161,99.76,49,292.15,757,0.24


In [20]:
df_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      50 non-null     object 
 1   TotalRank  50 non-null     int64  
 2   TotalSqMi  50 non-null     float64
 3   TotalKmQ   50 non-null     int64  
 4   LandRank   50 non-null     int64  
 5   LandSqMi   50 non-null     float64
 6   LandKmQ    50 non-null     int64  
 7   LandPer    50 non-null     float64
 8   WaterRank  50 non-null     int64  
 9   WaterSqMi  50 non-null     float64
 10  WaterKmQ   50 non-null     int64  
 11  WaterPer   50 non-null     float64
dtypes: float64(5), int64(6), object(1)
memory usage: 4.8+ KB


In [21]:
df_area.describe()

,TotalRank,TotalSqMi,TotalKmQ,LandRank,LandSqMi,LandKmQ,LandPer,WaterRank,WaterSqMi,WaterKmQ,WaterPer
count,50.00000,50.000000,5.000000e+01,50.00000,50.000000,5.000000e+01,50.000000,50.00000,50.000000,50.000000,50.000000
mean,25.50000,75933.477600,1.966668e+05,25.50000,70636.887800,1.829487e+05,91.668400,25.50000,5296.590400,13718.160000,8.331600
std,14.57738,97327.423173,2.520769e+05,14.57738,85815.678218,2.222615e+05,10.244987,14.57738,14256.070622,36923.113729,10.244987
min,1.00000,1544.890000,4.001000e+03,1.00000,1033.810000,2.678000e+03,58.460000,1.00000,191.830000,497.000000,0.240000
25%,13.25000,37416.612500,9.690850e+04,13.25000,36741.167500,9.515900e+04,87.222500,13.25000,705.762500,1828.000000,1.645000
50%,25.50000,57093.180000,1.478705e+05,25.50000,53891.280000,1.395775e+05,96.695000,25.50000,1501.210000,3888.000000,3.305000
75%,37.75000,84564.897500,2.190222e+05,37.75000,81225.725000,2.103735e+05,98.355000,37.75000,4373.040000,11325.750000,12.777500
max,50.00000,665384.040000,1.723337e+06,50.00000,570640.950000,1.477953e+06,99.760000,50.00000,94743.100000,245384.000000,41.540000


In [22]:
# Remove rank columns
df_area = remove_rank_cols(df_area)

# Remove invalid characters and replace whitespace in States with underscore
state_list = df_area['State'].tolist()
df_area['State'] = clean_states_text(state_list)

# Index and sort by State
df_area = index_sort(df_area)

In [23]:
df_area.head()

,TotalSqMi,TotalKmQ,LandSqMi,LandKmQ,LandPer,WaterSqMi,WaterKmQ,WaterPer
State,,,,,,,,
Alabama,52420.07,135767,50645.33,131171,96.61,1774.74,4597,3.39
Alaska,665384.04,1723337,570640.95,1477953,85.76,94743.10,245384,14.24
Arizona,113990.30,295234,113594.08,294207,99.65,396.22,1026,0.35
Arkansas,53178.55,137732,52035.48,134771,97.85,1143.07,2961,2.15
California,163694.74,423967,155779.22,403466,95.16,7915.52,20501,4.84
